# Test Table Chunkers

In [1]:
%load_ext autoreload
%autoreload 2

In [107]:
from pathlib import Path
import json
import pandas as pd
import sys
from typing import Any, Dict, Iterable, List, Sequence, Tuple, Union
import requests


In [3]:
root = Path.cwd().parent  # if you're inside notebooks/
sys.path.append(str(root / "src"))

In [5]:
def load_table_chunks(path: str | Path):
  """Load table chunks from a *.tables.jsonl file and rebuild DataFrames."""
  path = Path(path)
  chunks = []
  with path.open("r", encoding="utf-8") as f:
      for line in f:
          line = line.strip()
          if not line:
              continue
          obj = json.loads(line)
          tbl = obj.get("table_dict")
          if isinstance(tbl, dict) and "columns" in tbl and "data" in tbl:
              try:
                  obj["table_df"] = pd.DataFrame(**tbl)  # columns/index/data
              except Exception:
                  pass
          chunks.append(obj)
  return chunks

def load_aapl_10k_2024_tables():
  """Convenience loader for AAPL 10-K table chunks."""
  path = Path("../data/chunked/AAPL_10-K_2024.tables.jsonl")
  if not path.is_file():
      raise FileNotFoundError(f"File not found: {path}")
  return load_table_chunks(path)

In [6]:
def test_table_summary(table_chunk, api_url: str, model: str = "minimax-m2:cloud", temperature: float = 0.0):
  """
  Call the tables_summarizer LLM on a single table_chunk and print the JSON result.

  table_chunk: one element from the list returned by load_aapl_10k_2024_tables()
  api_url: your LLM HTTP endpoint, e.g. "http://localhost:11434/api/generate"
  """
  annotation = summarize_table_with_llm(
      table_chunk=table_chunk,
      model=model,
      api_url=api_url,
      temperature=temperature,
  )
  print(json.dumps(annotation, indent=2, ensure_ascii=False))
  return annotation

In [101]:
def df_to_markdown_for_llm(df: pd.DataFrame) -> str:
    """Convert DataFrame to markdown (no index)."""
    return df.to_markdown(index=False)

SYSTEM_PROMPT = """
You are a careful financial reporting analyst.

You are given:
- A markdown table extracted from an SEC 10-K or 10-Q.
- Optional filing metadata.

Your job:
1. Summarize in plain English what the entire table is about.
2. Summarize what each data row (non-header, non-section row) represents.

RULES:
- Use ONLY information from the table and metadata.
- Do NOT invent reasons, trends, or comparisons. Avoid change words like “increased”, “decreased”, “higher”, “lower”, etc.
- Do NOT repeat numeric values from the table (amounts, counts, dollar figures, etc.).
- You MAY mention years or period labels (e.g., 2022, 2023, 2024, Q1, September), copying them exactly.
- Summarize by row, NOT by column.
- Output MUST be a single valid JSON object, nothing else, starting with '{' and ending with '}'.

ROW HANDLING:
- If a row’s first cell is only a year or period label (e.g., "2024", "2024.0", "Q1"), or the row is mostly years/periods, treat it as a period header row, not a data row. Do NOT create a row_summaries entry for it.
- The first row after “Table (first row is header):” is the header row. Do NOT create a summary for the header row.
- Some rows are section headers that group later rows (e.g., labels ending with ':' like "Americas:", "Europe:", or rows with a label but no numeric data).
  - Do NOT create summaries for section headers.
  - Treat a section header as context for the rows that follow it, until the next section header.
- For a data row under a section header, build row_label as: "<section> – <metric label>".
  - Example: section "Americas:" + metric "Net sales$" → "Americas – Net sales$".
- If there is no section header, use the natural row label (first column) as row_label.
- You MUST output exactly one element in row_summaries for EACH non-header, non-section data row, in order.
- row_index is a 0-based index over these data rows:
  - The first data row has row_index = 0, the next has 1, etc.

JSON FORMAT:
{
  "table_summary": "one or two sentences about the whole table",
  "row_summaries": [
    {
      "row_index": <0-based integer index of the data row>,
      "row_label": "<row label, including section + metric if applicable>",
      "description": "one concise sentence describing what this row represents"
    }
  ]
}

EXAMPLE (ILLUSTRATIVE ONLY):

Filing metadata (example):
- Company: Apple Inc.
- Form type: 10-K
- Fiscal year: 2024
- Section path: PART II > Note – Segment Information and Geographic Data

Example table (first row is header):

| 1                     | 2        | 3    | 4        | 5    | 6        |
|:----------------------|:---------|:-----|:---------|:-----|:---------|
| 2024                  | 2024.0   | 2023 | 2023.0   | 2022 | 2022.0   |
| Americas:             |          |      |          |      |          |
| Net sales$            | 167045.0 | $    | 162560.0 | $    | 169658.0 |
| Operating income$     | 67656.0  | $    | 60508.0  | $    | 62683.0  |
| Europe:               |          |      |          |      |          |
| Net sales$            | 101328.0 | $    | 94294.0  | $    | 95118.0  |
| Operating income$     | 41790.0  | $    | 36098.0  | $    | 35233.0  |

Example JSON response for this example table:

{
  "table_summary": "The table presents segment information for Apple Inc., showing net sales and operating income for geographic segments such as Americas and Europe for fiscal years 2024, 2023, and 2022.",
  "row_summaries": [
    {
      "row_index": 0,
      "row_label": "Americas – Net sales$",
      "description": "Net sales for the Americas segment across fiscal years 2024, 2023, and 2022."
    },
    {
      "row_index": 1,
      "row_label": "Americas – Operating income$",
      "description": "Operating income for the Americas segment across fiscal years 2024, 2023, and 2022."
    },
    {
      "row_index": 2,
      "row_label": "Europe – Net sales$",
      "description": "Net sales for the Europe segment across fiscal years 2024, 2023, and 2022."
    },
    {
      "row_index": 3,
      "row_label": "Europe – Operating income$",
      "description": "Operating income for the Europe segment across fiscal years 2024, 2023, and 2022."
    }
  ]
}

When you receive a NEW table:
- Follow all rules above.
- Use section headers only as context.
- Create one row_summaries entry per data row.
- Do not repeat numeric values from the table.
- Respond with exactly one valid JSON object and nothing else.
"""


# ---------------------------------------------------------------------
#  Prompt builder (from table_summarization_eval, lightly adapted)
# ---------------------------------------------------------------------

def build_user_prompt(table_chunk: Dict[str, Any]) -> str:
    df: pd.DataFrame = table_chunk["table_df"]
    meta: Dict[str, Any] = table_chunk.get("meta", {})
    table_md = df_to_markdown_for_llm(df)
    heading_path = meta.get("heading_path", table_chunk.get("heading_path", []))

    return f"""Filing metadata:
- Company: {meta.get("company_name")}
- Ticker: {meta.get("ticker")}
- CIK: {meta.get("cik")}
- Form type: {meta.get("form_type")}
- Fiscal year: {meta.get("fiscal_year")}
- Filing date: {meta.get("filing_date")}
- Item: {meta.get("item_id")} – {meta.get("item_title")}
- Section path: { ' > '.join(heading_path) }

Table (first row is header):
{table_md}

Remember:
- Use only info in the table + metadata.
- Do not compute new metrics or trends.
- Do not repeat numeric values from the table.
- Years like 2023/2024/2025 are allowed.
- Respond with JSON only.
- You must output exactly one valid JSON object.
"""


# ---------------------------------------------------------------------
#  LLM summarization call (Ollama-style HTTP API)
# ---------------------------------------------------------------------

def summarize_table_with_llm(
    table_chunk: Dict[str, Any],
    model: str,
    api_url: str,
    temperature: float = 0.0,
    timeout: float = 180.0,
) -> Dict[str, Any]:
    """
    Call an LLM to summarize a table with strict JSON output.

    The API is expected to be compatible with Ollama's /generate endpoint:
      POST api_url with JSON:
        {
          "model": model,
          "prompt": full_prompt,
          "stream": false,
          "format": "json",
          "options": {"temperature": temperature, "num_predict": 4096}
        }

    The response JSON should contain a 'response' field that is a JSON string
    matching the schema described in SYSTEM_PROMPT.
    """
    user_prompt = build_user_prompt(table_chunk)
    full_prompt = SYSTEM_PROMPT.strip() + "\n\n---\n\n" + user_prompt.strip()
    # print(full_prompt)
    payload = {
        "model": model,
        "prompt": full_prompt,
        "stream": False,
        "format": "json",
        "options": {"temperature": temperature, "num_predict": 4096},
    }

    resp = requests.post(api_url, json=payload, timeout=timeout)
    resp.raise_for_status()

    obj = resp.json()
    text = (obj.get("response") or "").strip()
    # print(obj)
    
    # Parse the JSON string returned by the model
    return json.loads(text)

In [61]:
model = "qwen2.5:7b"
# "qwen2.5:3b-instruct"
api_url = "http://localhost:11434/api/generate"

In [8]:
aapl_tables = load_aapl_10k_2024_tables()
len(aapl_tables)

48

In [9]:
aapl_tables[0]["section_title"]

'Purchases of Equity Securities by the Issuer and Affiliated Purchasers'

In [25]:
aapl_tables[0]["table_df"]

,1,2,3,4,5
0,Periods,Total Number of Shares Purchased,Average Price Paid Per Share,Total Number of Shares Purchased as Part of Pu...,Approximate Dollar Value ofShares That May Yet...
1,"June 30, 2024 to August 3, 2024:",,,,
2,Open market and privately negotiated purchases,35697,$224.11,35697,
3,"August 4, 2024 to August 31, 2024:",,,,
4,Open market and privately negotiated purchases,42910,$221.39,42910,
5,"September 1, 2024 to September 28, 2024:",,,,
6,Open market and privately negotiated purchases,33653,$222.86,33653,
7,Total,112260,,,$89074


In [85]:
# Assuming you already did: aapl_tables = load_aapl_10k_2024_tables()
one_table = aapl_tables[0]
ann = test_table_summary(one_table, api_url, model)

{
  "table_summary": "The table details Apple Inc.'s purchases of its own equity securities for different periods in fiscal year 2024, including the number of shares purchased, average price paid per share, and total dollar value under publicly announced plans or programs.",
  "row_summaries": [
    {
      "row_index": 0,
      "row_label": "June 30, 2024 to August 3, 2024 – Open market and privately negotiated purchases",
      "description": "Purchases of Apple Inc.'s shares during the period from June 30, 2024, to August 3, 2024, including the number of shares purchased and the average price paid per share."
    },
    {
      "row_index": 1,
      "row_label": "August 4, 2024 to August 31, 2024 – Open market and privately negotiated purchases",
      "description": "Purchases of Apple Inc.'s shares during the period from August 4, 2024, to August 31, 2024, including the number of shares purchased and the average price paid per share."
    },
    {
      "row_index": 2,
      "row_

In [29]:
ann = test_table_summary(aapl_tables[1], api_url, model)

{
  "table_summary": "The table shows the performance of Apple Inc., S&P 500 Index, and Dow Jones U.S. Technology Supersector Index from September 2019 to September 2024.",
  "row_summaries": [
    {
      "row_index": 0,
      "row_label": "September 2019",
      "description": "The first row lists the base year for comparison, with Apple Inc. at $100, S&P 500 Index at $100, and Dow Jones U.S. Technology Supersector Index at $100."
    },
    {
      "row_index": 1,
      "row_label": "Apple Inc.$",
      "description": "This row represents the performance of Apple Inc., with values for each month from September 2019 to September 2024."
    },
    {
      "row_index": 2,
      "row_label": "S&P 500 Index$",
      "description": "This row shows the performance of the S&P 500 Index, with values for each month from September 2019 to September 2024."
    },
    {
      "row_index": 3,
      "row_label": "Dow Jones U.S. Technology Supersector Index$",
      "description": "This row indicat

### Test Long Table 
#### section_title == "CONSOLIDATED STATEMENTS OF OPERATIONS"

In [22]:
CSOO_table = list(filter(lambda x:x['section_title'] == "CONSOLIDATED STATEMENTS OF OPERATIONS", aapl_tables))[0]
CSOO_table['table_df']

,1,2,3,4
0,,Years ended,Years ended,Years ended
1,,"September 28, 2024","September 30, 2023","September 24, 2022"
2,Net sales:,,,
3,Products,$294866,$298085,$316199
4,Services,96169,85200,78129
5,Total net sales,391035,383285,394328
6,Cost of sales:,,,
7,Products,185233,189282,201471
8,Services,25119,24855,22075
9,Total cost of sales,210352,214137,223546


In [86]:
CSOO_ann = test_table_summary(CSOO_table, api_url, model)

{
  "table_summary": "The table presents consolidated statements of operations for Apple Inc., showing various financial metrics such as net sales, cost of sales, gross margin, operating expenses, and net income for the fiscal years ending September 28, 2024, September 30, 2023, and September 24, 2022.",
  "row_summaries": [
    {
      "row_index": 0,
      "row_label": "Net sales: – Products",
      "description": "Products net sales for the fiscal years ending September 28, 2024, September 30, 2023, and September 24, 2022."
    },
    {
      "row_index": 1,
      "row_label": "Net sales: – Services",
      "description": "Services net sales for the fiscal years ending September 28, 2024, September 30, 2023, and September 24, 2022."
    },
    {
      "row_index": 2,
      "row_label": "Net sales: – Total net sales",
      "description": "Total net sales for the fiscal years ending September 28, 2024, September 30, 2023, and September 24, 2022."
    },
    {
      "row_index": 3,
  

#### Test Segregated Table

In [41]:
CIGD_table = list(filter(lambda x:x['section_title'] == "Note 13 – Segment Information and Geographic Data", aapl_tables))[0]
CIGD_table['table_df']

,1,2,3,4,5,6
0,2024,2024.0,2023,2023.0,2022,2022.0
1,Americas:,,,,,
2,Net sales$,167045.0,$,162560.0,$,169658.0
3,Operating income$,67656.0,$,60508.0,$,62683.0
4,Europe:,,,,,
5,Net sales$,101328.0,$,94294.0,$,95118.0
6,Operating income$,41790.0,$,36098.0,$,35233.0
7,Greater China:,,,,,
8,Net sales$,66952.0,$,72559.0,$,74200.0
9,Operating income$,27082.0,$,30328.0,$,31153.0


In [87]:
CIGD_ann = test_table_summary(CIGD_table, api_url, model)

{
  "table_summary": "The table presents segment information for Apple Inc., showing net sales and operating income for geographic segments such as Americas, Europe, Greater China, Japan, and Rest of Asia Pacific for fiscal years 2024, 2023, and 2022.",
  "row_summaries": [
    {
      "row_index": 0,
      "row_label": "Americas – Net sales$",
      "description": "Net sales for the Americas segment across fiscal years 2024, 2023, and 2022."
    },
    {
      "row_index": 1,
      "row_label": "Americas – Operating income$",
      "description": "Operating income for the Americas segment across fiscal years 2024, 2023, and 2022."
    },
    {
      "row_index": 2,
      "row_label": "Europe – Net sales$",
      "description": "Net sales for the Europe segment across fiscal years 2024, 2023, and 2022."
    },
    {
      "row_index": 3,
      "row_label": "Europe – Operating income$",
      "description": "Operating income for the Europe segment across fiscal years 2024, 2023, and 202

## Try running on all tables

In [94]:
temperature = 0.0

for i, table_chunk in enumerate(aapl_tables):
    print(f"\n==== TABLE {i} ====")
    print(f"Approx rows: {len(table_chunk.get('table_df', []))}")  # or however you store it

    try:
        annotation = summarize_table_with_llm(
            table_chunk=table_chunk,
            model=model,
            api_url=api_url,
            temperature=temperature,
        )
    except Exception as e:
        print(f"ERROR on table {i}: {e}")
        continue


==== TABLE 0 ====
Approx rows: 8

==== TABLE 1 ====
Approx rows: 4

==== TABLE 2 ====
Approx rows: 7

==== TABLE 3 ====
Approx rows: 7

==== TABLE 4 ====
Approx rows: 5

==== TABLE 5 ====
Approx rows: 4

==== TABLE 6 ====
Approx rows: 7

==== TABLE 7 ====
Approx rows: 4

==== TABLE 8 ====
Approx rows: 3

==== TABLE 9 ====
Approx rows: 8

==== TABLE 10 ====
Approx rows: 26

==== TABLE 11 ====
Approx rows: 15

==== TABLE 12 ====
Approx rows: 36
ERROR on table 12: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=120.0)

==== TABLE 13 ====
Approx rows: 22

==== TABLE 14 ====
Approx rows: 37

==== TABLE 15 ====
Approx rows: 7

==== TABLE 16 ====
Approx rows: 9

==== TABLE 17 ====
Approx rows: 18

==== TABLE 18 ====
Approx rows: 18

==== TABLE 19 ====
Approx rows: 6

==== TABLE 20 ====
Approx rows: 4

==== TABLE 21 ====
Approx rows: 7

==== TABLE 22 ====
Approx rows: 4

==== TABLE 23 ====
Approx rows: 4

==== TABLE 24 ====
Approx rows: 4

==== TABLE 25 ====
Ap

In [99]:
test_table_12 = aapl_tables[12]
test_table_summary(test_table_12, api_url, model)

{
  "table_summary": "The table presents consolidated balance sheet information for Apple Inc., comparing assets, liabilities, and shareholders’ equity as of September 28, 2024, and September 30, 2023.",
  "row_summaries": [
    {
      "row_index": 0,
      "row_label": "ASSETS: – Current assets",
      "description": "Current assets for the balance sheet as of September 28, 2024, and September 30, 2023."
    },
    {
      "row_index": 1,
      "row_label": "Cash and cash equivalents",
      "description": "Cash and cash equivalents for the balance sheet as of September 28, 2024, and September 30, 2023."
    },
    {
      "row_index": 2,
      "row_label": "Marketable securities",
      "description": "Marketable securities for the balance sheet as of September 28, 2024, and September 30, 2023."
    },
    {
      "row_index": 3,
      "row_label": "Accounts receivable, net",
      "description": "Accounts receivable, net for the balance sheet as of September 28, 2024, and September

{'table_summary': 'The table presents consolidated balance sheet information for Apple Inc., comparing assets, liabilities, and shareholders’ equity as of September 28, 2024, and September 30, 2023.',
 'row_summaries': [{'row_index': 0,
   'row_label': 'ASSETS: – Current assets',
   'description': 'Current assets for the balance sheet as of September 28, 2024, and September 30, 2023.'},
  {'row_index': 1,
   'row_label': 'Cash and cash equivalents',
   'description': 'Cash and cash equivalents for the balance sheet as of September 28, 2024, and September 30, 2023.'},
  {'row_index': 2,
   'row_label': 'Marketable securities',
   'description': 'Marketable securities for the balance sheet as of September 28, 2024, and September 30, 2023.'},
  {'row_index': 3,
   'row_label': 'Accounts receivable, net',
   'description': 'Accounts receivable, net for the balance sheet as of September 28, 2024, and September 30, 2023.'},
  {'row_index': 4,
   'row_label': 'Vendor non-trade receivables',
 

In [102]:
t = test_table_summary(aapl_tables[45], api_url, model)

{
  "table_summary": "The table lists various exhibits and their descriptions, including their exhibit numbers, descriptions, forms of incorporation, and filing dates or period end dates for Apple Inc.'s Form 10-K for fiscal year 2024.",
  "row_summaries": [
    {
      "row_index": 0,
      "row_label": "Exhibit Number – Exhibit Description",
      "description": "Lists the exhibit number and description of various documents, including their forms of incorporation and filing dates."
    },
    {
      "row_index": 1,
      "row_label": "4.28 – Officer’s Certificate of the Registrant, dated as of May 10, 2023",
      "description": "Provides an officer's certificate for Apple Inc., including forms of global notes representing certain bond issues."
    },
    {
      "row_index": 2,
      "row_label": "4.29* – Apple Inc. Deferred Compensation Plan",
      "description": "Describes the deferred compensation plan for Apple Inc., filed under S-8 form on August 23, 2018."
    },
    {
     

## Test table summarizer function

## Test Parallellization

In [111]:
import time
from pathlib import Path
from typing import Any, Dict, List, Optional

from concurrent.futures import ThreadPoolExecutor, as_completed
from rag10kq.tables_summarizer import summarize_prefix
from rag10kq.tables_summarizer import load_table_chunks_for_prefix

In [113]:
prefix = "AAPL_10-K_2024"
chunks_dir = Path("../data/chunked")  # chunk_dir for AAPL 10-K 2024

In [105]:
def _summarize_single_table(
    prefix: str,
    table_index: int,
    table_chunk: Dict[str, Any],
    api_url: str,
    model: str,
    temperature: float,
) -> Dict[str, Any]:
    """
    Internal helper: summarize one table_chunk and return a record dict.
    Safe to call from multiple threads.
    """
    start = time.time()
    error: Optional[str] = None
    annotation: Optional[Dict[str, Any]] = None

    try:
        annotation = summarize_table_with_llm(
            table_chunk=table_chunk,
            model=model,
            api_url=api_url,
            temperature=temperature,
        )
    except Exception as exc:
        error = str(exc)
        print(f"[ERROR] prefix={prefix} table_index={table_index}: {error}")

    elapsed = time.time() - start

    rec: Dict[str, Any] = {
        "prefix": prefix,
        "table_index": table_index,
        "section_title": table_chunk.get("section_title"),
        "model": model,
        "elapsed_sec": elapsed,
        "error": error,
        "annotation": annotation,
    }
    return rec

In [115]:
def summarize_prefix_parallel(
    prefix: str,
    chunks_dir: Path,
    api_url: str,
    model: str,
    temperature: float,
    max_tables: Optional[int] = None,
    max_workers: int = 3,
) -> List[Dict[str, Any]]:
    """
    Summarize all (or up to max_tables) tables for a single filing prefix,
    using a thread pool to parallelize LLM calls.

    max_workers controls how many tables are processed concurrently.
    For a local LLM (e.g., qwen2.5:7b on a laptop), 2–3 is usually a safe range.
    """
    start_all = time.time()
    table_chunks = load_table_chunks_for_prefix(prefix, chunks_dir)

    # Respect max_tables by slicing the list of chunks
    if max_tables is not None:
        table_chunks = table_chunks[:max_tables]

    records: List[Dict[str, Any]] = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all jobs
        future_to_index = {}
        for idx, table_chunk in enumerate(table_chunks):
            fut = executor.submit(
                _summarize_single_table,
                prefix,
                idx,
                table_chunk,
                api_url,
                model,
                temperature,
            )
            future_to_index[fut] = idx

        # Collect results as they complete
        for fut in as_completed(future_to_index):
            rec = fut.result()
            records.append(rec)

    # Ensure records are sorted by table_index to match original order
    records.sort(key=lambda r: r["table_index"])
    total_elapsed = time.time() - start_all
    print(
        f"[SUMMARY] prefix={prefix} "
        f"tables={len(table_chunks)} "
        f"total_elapsed={total_elapsed:.2f}s (sequential)"
    )
    return records